**Importamos las librerías a utilizar**

In [0]:
#Importamos la libreria de pandas y numpy
import pandas as pd
import numpy as np

**Importando datos de Toronto de Wikipedia**


In [137]:
ruta = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
listar = pd.read_html(ruta)
df_toronto = listar[0]
import numpy as np
df_toronto['Borough'].replace('Not assigned',np.nan,inplace = True)
df_toronto.dropna(subset=['Borough'], axis = 0, inplace = True)
df_toronto.reset_index(inplace=True)
df_toronto.drop(columns = {'index'},inplace = True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Aqui vamos a resolver el punto:**

Puede existir más de un vecindario en un área de código postal. 
Por ejemplo, en la tabla de la página de Wikipedia, notará que M5A aparece dos veces y tiene dos vecindarios: 
Harbourfront y Regent Park . Estas dos filas se combinarán en una fila con los barrios separados con una coma como se muestra en la fila 11 de la 
tabla anterior.

In [138]:
#Validamos si existen duplicados
df2 = df_toronto['Postal Code'].duplicated()
df2.value_counts()#Hay 103 resultados como False lo que indica que no hay duplicados

False    103
Name: Postal Code, dtype: int64

In [139]:
#Nos cercioramos de todos modos con el Cod Postal que ponen como ejemplo M5A
df2 = df_toronto[df_toronto['Postal Code']=='M5A']
df2 # Como se observa solo nos devuelve un registro

,Postal Code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Si una celda tiene un distrito pero no un vecindario asignado , el vecindario será el mismo que el distrito.**

In [140]:
df_T = df_toronto[df_toronto['Neighborhood']=='Not assigned']
df_T #Vemos que no ay vencidarios como Not assigned

,Postal Code,Borough,Neighborhood
